In [31]:
import numpy as np #1.24.4
import os
import cv2 #4.9.0
from matplotlib import pyplot as plt
import pickle as pkl
import time as t
np.random.seed(0)

In [32]:
input_dataset = os.path.join(os.path.dirname(os.getcwd()), 'datasets')
#input_dataset
medicine_list = os.listdir(input_dataset)
#medicine_list

In [ ]:
a = np.array(cv2.resize(cv2.imread('c:/Users/PC/OneDrive/Desktop/MedicurePlus/Backend/datasets/Aspirin/1.png', cv2.IMREAD_GRAYSCALE), (128, 128)).flat)
b = np.array(cv2.resize(cv2.imread('c:/Users/PC/OneDrive/Desktop/MedicurePlus/Backend/datasets/Aspirin/2.png', cv2.IMREAD_GRAYSCALE), (128, 128)).flat)
inp_vec = np.array([a, b])

In [ ]:
plt.imshow(cv2.imread('c:/Users/PC/OneDrive/Desktop/MedicurePlus/Backend/datasets/Aspirin/10.png'), cmap='gray')

In [33]:
image_input_list = []
image_output_list = []
for dirpath, dirnames, filenames in os.walk(input_dataset):
    for filename in filenames:
        path = os.path.join(dirpath, filename)
        if(os.path.isfile(path=path)):
            image_input_list.append(np.array(cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (128, 128)).flat))
            image_output_list.append(dirpath.split('\\')[-1])

In [ ]:
class NeuralNetwork:
    def __init__(self) -> None:
        self.layers = []
        self.biases = []
        self.iterator = []
    
    def addLayer(self, *args) -> None:
        randomWeights = 0.01 * np.random.randn(*args)
        biasLayer = 0.01 * np.random.randn(args[0], 1)
        self.layers.append(randomWeights)
        self.biases.append(biasLayer)

    def RELU(self, inp):
        return np.maximum(0, inp)
    
    def SoftMax(self):
        exp_vals = np.exp(self.iterator[-1] - np.max(self.iterator[-1], axis=0, keepdims=True))
        self.predicted = exp_vals / np.sum(exp_vals, axis=0, keepdims=True)
    
    def forward(self, inp : list[any]):
        self.clearIt()
        self.iterator.append(np.array(inp).T)
        for layer, bias in zip(self.layers, self.biases):
            self.iterator.append(self.RELU(np.dot(layer, self.iterator[-1]) + bias))
        self.SoftMax()
        return self.predicted
    
    def calculateLoss(self):
        self.loss =  -np.sum(np.array(self.output_mat) * np.log(self.predicted + 1e-10), axis=0, keepdims=True)
        return self.loss
    
    def calculateCost(self):
        self.cost = np.mean(self.calculateLoss())
        return self.cost
    
    def train(self, actual_inputs, classlist, actual_ouputs, epoach : int = 1):
        self.encodedOuput(classlist=classlist, actual_output=actual_ouputs)
        while(epoach > 0):
            self.forward(actual_inputs)
            self.backProp()
            print(self.calculateCost())
            epoach -= 1

    def backProp(self):
    # Calculate delta for the last layer (output layer)
        delta = self.predicted - self.output_mat  # delta for output layer
        grads_w = []
        grads_b = []
        
        # Gradient for the last layer's weights
        grad_w_L = np.dot(delta, self.iterator[-2].T)  # Layer 3 weight gradient
        grads_w.append(grad_w_L)
        
        # Gradient for the last layer's bias
        grad_b_L = np.sum(delta, axis=1, keepdims=True)
        grads_b.append(grad_b_L)

        # Update deltas for earlier layers
        next_delta = delta
        for l in range(len(self.layers) - 2, -1, -1):  # going backward through layers
            # Gradient for weights and biases
            next_delta = np.dot(self.layers[l + 1].T, next_delta) * (self.iterator[l + 1] > 0).astype(float)  # ReLU derivative
            grad_w = np.dot(next_delta, self.iterator[l].T)
            grad_b = np.sum(next_delta, axis=1, keepdims=True)
            
            grads_w.append(grad_w)
            grads_b.append(grad_b)
        
        # Apply gradients in reverse order (to align with layer order)
        grads_w = grads_w[::-1]
        grads_b = grads_b[::-1]

        for i in range(len(self.layers)):
            self.layers[i] -= self.learningRate * grads_w[i]
            self.biases[i] -= self.learningRate * grads_b[i]

        
    def setLearningRate(self, rate):
        self.learningRate = rate

    def encodedOuput(self, classlist, actual_output):
        self.output_mat = np.equal(np.array(classlist).reshape(len(classlist), 1), np.array(actual_output).reshape(1, len(actual_output))).astype(float)
        return self.output_mat
    
    def clearIt(self):
        self.iterator.clear()
    
    def reset(self):
        self.layers.clear()
        self.biases.clear()
        self.iterator.clear()


In [ ]:
network = NeuralNetwork()
network.addLayer(128, 128 * 128)
network.addLayer(258, 128)
network.addLayer(len(medicine_list), 258)
network.setLearningRate(0.01)

In [ ]:
network.train(image_input_list, medicine_list, image_output_list, 200)

In [ ]:
network.setLearningRate(0.1)

In [ ]:
with open('imageClassiferModel.pkl', 'wb') as f:
    pkl.dump(network, f)

In [ ]:
with open('imageClassiferModel.pkl', 'rb') as f:
    imageClassifer : NeuralNetwork = pkl.load(f)
    imageClassifer.setLearningRate(0.00001)
    maxoutput = np.argmax(imageClassifer.forward(image_input_list), axis=0)
print(maxoutput)

In [25]:
import easyocr as eo

In [ ]:
class ImageClassifier:
    class Trie:
        class Node:
            def __init__(self, end : bool = False) -> None:
                self.end = end
                self.children = [None for i in range(38)]
                
        def __init__(self) -> None:
            self.node = self.Node()
            self.__similarityList = []
        
        def encrypt(self, ch : chr) -> int:
            if(ch >= 'a' and ch <= 'z'):
                return ord(ch) - ord('a')
            elif(ch >= '0' and ch <= '9'):
                return ord(ch) - ord('0') + 26
            elif(ch == ' '):
                return 36
            elif(ch == '-'):
                return 37
        
        def decrypt(self, ind : int) -> chr:
            if(ind >= 0 and ind <= 25):
                return chr(ind + ord('a'))
            elif(ind >= 26 and ind <= 65):
                return chr(ind - 26 + ord('0'))
            elif(ind == 36):
                return ' '
            elif(ind == 37):
                return '-'

        def add(self, item : str):
            item = item.lower()
            cur = self.node
            for index, letter in enumerate(item):
                if(cur.children[(self.encrypt(letter))] != None):
                    cur = cur.children[self.encrypt(letter)]
                else:
                    cur.children[self.encrypt(letter)] = self.Node(end = (index == len(item) - 1))
                    cur = cur.children[self.encrypt(letter)]

        def showSimilarities(self, item : str):
            cur = self.node
            for letter in item:
                if(cur.children[(self.encrypt(letter))] != None):
                    cur = cur.children[self.encrypt(letter)]
            
            self.__recursiveSearch(cur=cur, prev=item)
            return self.__similarityList

        def __recursiveSearch(self, cur : Node, prev : str = ''):
            if(cur.end):
                self.__similarityList.append(prev)
            for index, node in enumerate(cur.children):
                if(node is not None):
                    self.__recursiveSearch(node, (prev + self.decrypt(index)))

    def __init__(self) -> None:
        self.__reader = eo.Reader(['en'])
        self.__trie = self.Trie()
    
    def read(self, image):
        self.matches = self.__reader.readtext(image=image)

    def addClass(self, classInput : str | list[str]):
        if(isinstance(classInput, str)):
            self.__trie.add(classInput)
            return
        for cls in classInput:
            self.__trie.add(cls)

    def findSimilarity(self) -> list[str]:
        pass

In [36]:
imageClassifier = ImageClassifier()
imageClassifier.addClass(medicine_list)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


AttributeError: 'ImageClassifier' object has no attribute '__trie'